sales = unit of sales

revenue = amount of money

Original plan:

revenue at each store ~ everything we can get

From the original dataset, we will:

1. manipulate the dataframe to the way we desire.

2. combine daily usales to weekly sales, pair each week with events (joined from calendar.csv).

3. pair each weekly sales with its correspondance price (joined from sales_price.csv).

4. sales * price = weekly revenue for the item.

5. add all revenues from the same store for the total revenue of that store.

for each data point, we will have: revenue, store_id, week_id, and event .

In this case, dependent var will be the revenue, independent var will be the rest of them. We will have store_id and event as categorical and create dummy var for them (9 + 4 cols). Note that we have to keep all four columns for event because there is a NaN type. We will treat week_id as numerical because there are 278 different values. 

The size of training data will be 278 weeks * 10 stores = 2780 rows and 9+4+1+1=15 columns (2780, 15).

The size of testing data will be 4 weeks * 10 stores = 40 rows.

My concern is:

1. weekly_id has cyclic effect as sales at the same period per year will be similar. For example, sales will always boost during holiday seasons and drop down a bit after that. 

2. it is not true that as time goes on, the revenue will have a upward or downward trend over the period of time of the dataset.

Intuitively, our numerical input is not linear to the output.


My plan proposed during the meeting is:

sale (in unit) ~ everything else

everything else = price + department_id + store_id + weekly_id + event

From the original dataset:

1. manipulate the dataframe to the way we desire.

2. combine daily usales to weekly sales, pair each week with events (joined from calendar.csv)

3. pair each weekly sales with its correspondance price (joined from sales_price.csv)

And that's it. It is actualy easier than the first plan because for the first plan, to get the revenue, we have to visit price no matter how.

for each data point, we will have: sales, store_id, department_id, week_id, and event.

In this case, dependent var will be the sales, independent var will be the rest of them. We will have store_id, department_id and event as categorical and create dummy var for them (9 + 6 + 4 cols). We will treat week_id as numerical or categorical (not sure yet).

The size of training data will be 278 weeks * 3049 items * 10 stores = 8476220 rows and 9+6+4+1+1=21 columns (8476220, 21).

The size of testing data will be 4 weeks * 3049 items * 10 stores = 121960 rows.

The benefit of doing this is:

1. price is has linear relationship with sales.

2. data preprocessing load is no higher than the first plan.

3. more rows could lead to more accurate model (maybe).


In [ ]:
278*3049*10

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Set working directory
import sys, os
os.chdir(sys.path[0])

sys.path[0]

In [ ]:
df = pd.read_csv('Data/sales_train_evaluation.csv')

In [ ]:
df.head()

In [ ]:
df_day = df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
        var_name='day', 
        value_name='sales')

In [ ]:
df_day.head()

In [ ]:
df_day_copy = df_day.copy()
# df_day = df_day_copy.copy()

In [ ]:
df_day.shape

In [ ]:
30490*1941

In [ ]:
df_day.drop(['id','cat_id','state_id'], axis=1, inplace=True)

In [ ]:
df_day.head()

In [ ]:
df_day['day'] = df_day['day'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
df_day['week'] = df_day['day']//7+1

In [ ]:
df_day['day']

In [ ]:
df_day['week']

In [ ]:
df_day.head()

From here we can join df_cal with df_day by column 'd' to get the wm_yr_wk and event. After that, we can do something like below to get the weekly sales. With 'wm_yr_wk', 'item_id', 'store_id' we can get price from sell_prices.csv.

after that item_id will be replaced by price in df_week so we have our num var.

Things to pay attention: 

1. find a way to aggregate event weekly.

2. the numerical week is the one I mentioned to be treated as input. We need somehow find a way to link it with wm_yr_week to make sure it is accurate. Right now I think it needs calibration.

In [ ]:
df_cal = pd.read_csv('Data/calendar.csv')

In [ ]:
df_cal['d'] = df_cal['d'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
df_cal.shape

In [ ]:
df_cal.head(20)

It seems that for col 'wm_yr_wk', the first three digits is the year, the last two is the week.

In [ ]:
df_event = df_cal.loc[:,['wm_yr_wk', 'event_type_1', 'event_type_2']]
df_event.head()

In [ ]:
df_event = pd.get_dummies(df_event)
df_event.head()

In [ ]:
df_event = df_event.groupby(['wm_yr_wk'], as_index=False).sum()

In [ ]:
df_event.head()

In [ ]:
df_event.describe()

In [ ]:
(df_event.event_type_1_Religious == 2).sum()

In [ ]:
df_day = df_day.merge(df_cal.loc[:,['wm_yr_wk','d']],left_on='day',right_on='d')

In [ ]:
df_day.drop(['day','d'],axis=1,inplace=True)
df_day.rename(columns={'week':'week_id'},inplace=True)

In [ ]:
df_day.head()

In [ ]:
df_week = df_day.groupby(['item_id','wm_yr_wk','dept_id','store_id'], as_index=False).sum()

In [ ]:
df_week_copy = df_week.copy()
#df_week = df_week_copy.copy()

In [ ]:
df_week.drop(['week_id'],axis=1,inplace=True)

In [ ]:
df_week.shape

In [ ]:
df_week.head(10)

In [ ]:
278*30490

In [ ]:
df_price = pd.read_csv('Data/sell_prices.csv')

In [ ]:
df_price.head()

In [ ]:
df_price.shape

In [ ]:
df_week = df_week.merge(df_price, how='left' ,left_on=['store_id', 'item_id', 'wm_yr_wk'], right_on=['store_id', 'item_id', 'wm_yr_wk'])

In [ ]:
df_week = df_week.merge(df_event, how='left', left_on='wm_yr_wk', right_on='wm_yr_wk')

In [ ]:
df_week.head(10)

In [ ]:
df_week.shape

Aggregate by store:

In [ ]:
df_week['revenue'] = df_week.apply(lambda x: x.sales * x.sell_price, axis=1)

In [ ]:
df_week.head(10)

In [ ]:
df_week['sell_price'].isna().sum()

In [ ]:
df_week['sales'].isna().sum()

In [ ]:
((df_week['sell_price'].isna()) & (df_week['sales'] != 0)).sum()

In [ ]:
df_week_store = df_week.drop(['item_id', 'dept_id', 'sales', 'sell_price'], axis=1)
df_week_store.head(10)

In [ ]:
df_week_store = df_week_store.groupby(['store_id','wm_yr_wk'], as_index=False).sum()

In [ ]:
df_week_store.shape

In [ ]:
df_week_store.head()

In [ ]:
df_week_store.describe()

In [ ]:
df_week_store[['event_type_1_Cultural', 'event_type_1_National', 'event_type_1_Religious', 'event_type_1_Sporting', 'event_type_2_Cultural', 'event_type_2_Religious']] = \
(df_week_store[['event_type_1_Cultural', 'event_type_1_National', 'event_type_1_Religious', 'event_type_1_Sporting', 'event_type_2_Cultural', 'event_type_2_Religious']]\
 /3049).astype(int)

In [ ]:
df_week_store.store_id.nunique()

In [ ]:
df_dummy = pd.get_dummies(df_week_store['store_id'], prefix='store_id').iloc[:,:-1]

In [ ]:
df_week_store = pd.concat([df_week_store.iloc[:,1:], df_dummy], axis=1)

In [ ]:
df_week_store.head(10)

In [ ]:
df_week_store.shape

In [ ]:
df_week_store.isna().sum()

In [1]:
import pandas_datareader as pdr

start_date = '2011-01-29'
end_date = '2016-06-19' 

# Request data via Yahoo public API
data = pdr.get_data_yahoo('AMZN', start_date, end_date)

# Display Info
print(data.info())

data['Adj Close']

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1355 entries, 2011-01-31 to 2016-06-17
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       1355 non-null   float64
 1   Low        1355 non-null   float64
 2   Open       1355 non-null   float64
 3   Close      1355 non-null   float64
 4   Volume     1355 non-null   int64  
 5   Adj Close  1355 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 74.1 KB
None


Date
2011-01-31     8.482000
2011-02-01     8.605500
2011-02-02     8.676500
2011-02-03     8.685500
2011-02-04     8.796500
                ...    
2016-06-13    35.762001
2016-06-14    35.965000
2016-06-15    35.713001
2016-06-16    35.875500
2016-06-17    35.319500
Name: Adj Close, Length: 1355, dtype: float64

In [5]:
len(data)

1355